# Install and Import Required Libraries
Install the `datasets` library if not already installed and import the necessary modules.

In [2]:
# Install the datasets library if not already installed
# !pip install datasets

# Import necessary modules
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
import os

/Users/ignazioemanuelepicciche/Documents/Ignazio PC/ucbm/deep_learning/NLP_Generative_Task_QnA/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the Dataset
Use the `datasets` library to fetch and load the dataset from the Hugging Face Hub.

In [3]:
# Load the dataset from the Hugging Face Hub
dataset_name = "enelpol/rag-mini-bioasq"  # Replace with the desired dataset name
dataset = load_dataset(dataset_name, 'question-answer-passages')

# Split the Dataset into Train, Test, and Validation
Split the dataset into training, testing, and validation subsets using the `train_test_split` method or similar functionality.

In [4]:
# Split the train dataset into train and validation subsets; keep test unchanged

train_validation_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_validation_split["train"]
validation_dataset = train_validation_split["test"]
test_dataset = dataset["test"]


# Create a DatasetDict
Combine the train, test, and validation subsets into a `DatasetDict` object for further processing.

In [5]:
# Combine the subsets into a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "validation": validation_dataset
})

# Display the structure of the DatasetDict
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'id', 'relevant_passage_ids'],
        num_rows: 3209
    })
    test: Dataset({
        features: ['question', 'answer', 'id', 'relevant_passage_ids'],
        num_rows: 707
    })
    validation: Dataset({
        features: ['question', 'answer', 'id', 'relevant_passage_ids'],
        num_rows: 803
    })
})


# Save the Raw DatasetDict
Save the unprocessed `DatasetDict` to disk for future use.

In [6]:
os.makedirs("data/prepared_data", exist_ok=True)  # Create directory if it doesn't exist
dataset_dict.save_to_disk("data/prepared_data")  # Save the dataset to disk
print("Raw dataset saved to data/prepared_data")

Saving the dataset (1/1 shards): 100%|██████████| 803/803 [00:00<00:00, 219644.33 examples/s]

Raw dataset saved to data/prepared_data


# Preprocess the Dataset for Model Training
Tokenize the dataset for use with the T5 model. This step is crucial for fine-tuning as it converts the text data into the format required by the model.

In [7]:
# Define the model name for tokenization
model_name = "google-t5/t5-small"  # This should match the model in .env file

# Load the tokenizer for the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
# Define the preprocessing function
def preprocess_function(examples):
    # Add a prefix to the questions as T5 is trained with task prefixes
    prefix = "answer the question: "
    inputs = [prefix + q for q in examples["question"]]
    
    # Tokenize the inputs (questions) with padding and truncation
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    
    # Tokenize the targets (answers) to create the labels
    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True, padding="max_length")
    
    # Add the labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [9]:
# Apply the preprocessing function to the dataset
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

# Remove the original columns as they are no longer needed by the model
tokenized_datasets = tokenized_datasets.remove_columns(["question", "answer", "id", "relevant_passage_ids"])

# Display the structure of the tokenized dataset
print(tokenized_datasets)

Map: 100%|██████████| 803/803 [00:00<00:00, 6393.10 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3209
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 707
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 803
    })
})


# Save the Tokenized Dataset
Save the preprocessed and tokenized `DatasetDict` to disk for use with the model training script.

In [10]:
# Create directory for the tokenized dataset
tokenized_dataset_path = "data/tokenized_data"
os.makedirs(tokenized_dataset_path, exist_ok=True)

# Save the tokenized dataset to disk
tokenized_datasets.save_to_disk(tokenized_dataset_path)
print(f"Tokenized dataset saved to {tokenized_dataset_path}")

Saving the dataset (1/1 shards): 100%|██████████| 803/803 [00:00<00:00, 110575.73 examples/s]

Tokenized dataset saved to data/tokenized_data


# Update the .env File (Optional)
If you need to use the tokenized dataset for training, you may want to update the .env file to point to this new dataset.

In [11]:
# This is a helper code to update the .env file if needed
# Uncomment and run if you want to automatically update the .env file

# import os
# from dotenv import load_dotenv, find_dotenv, set_key

# dotenv_path = find_dotenv()
# load_dotenv(dotenv_path)

# absolute_tokenized_path = os.path.abspath(tokenized_dataset_path)
# set_key(dotenv_path, "TOKENIZED_DATASET", absolute_tokenized_path)

# print(f"Updated .env file with TOKENIZED_DATASET={absolute_tokenized_path}")